In [13]:
import pandas as pd

# EDA

In [14]:
from data_types import Author, AuthorsDB
from data_types import Publication, PublicationsDB
from data_types import Abstract, AbstractsDB
audb = AuthorsDB()
audb.load()
pubdb = PublicationsDB()
pubdb.load()
absdb = AbstractsDB()
absdb.load()


In [15]:
print((absdb.db['abstract'].isnull()).sum())
print((absdb.db['keywords'].isnull()).sum())

96
313


In [16]:
print((absdb.db.isnull()).sum())

abstract     96
keywords    313
dtype: int64


In [17]:
absdb.db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 586 entries, vyurv213 to tmf159
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   abstract  490 non-null    object
 1   keywords  273 non-null    object
dtypes: object(2)
memory usage: 13.7+ KB


In [18]:
pub = pd.DataFrame.from_dict(pubdb.db,orient='index')

In [19]:
pub.head()

,author_id,doi,udk,send,type,reference,by,paper,jour,yr,vol,issue,pages
vyurv213,"[113970, 148811, 22428]","<a class=""SLink"" href=""https://doi.org/10.1452...",004.052.3,20.11.2018,Статья,,"А.~А.~Бондаренко, П.~А.~Ляхов, М.~В.~Якобовский",Координированное сохранение с журналированием ...,Вестн. ЮУрГУ. Сер. Выч. матем. информ.,2019,8,2,76--91
vyurv1,"[113970, 22428]","<a class=""SLink"" href=""https://doi.org/10.1452...",004.052.3,13.04.2015,Статья,,"А.~А.~Бондаренко, М.~В.~Якобовский",Моделирование отказов в высокопроизводительных...,Вестн. ЮУрГУ. Сер. Выч. матем. информ.,2015,4,3,5--12
vyurv46,"[113970, 22428]",None,004.052.3,05.08.2014,Статья,,"А.~А.~Бондаренко, М.~В.~Якобовский",Обеспечение отказоустойчивости высокопроизводи...,Вестн. ЮУрГУ. Сер. Выч. матем. информ.,2014,3,3,20--36
ipmp2988,"[25339, 26426, 31779, 26427]",:https://doi.org/10.20948/prepr-2021-71,None,None,Препринт,"С. В. Поляков, М. А. Трапезникова, А. Г. Чурба...","С.~В.~Поляков, М.~А.~Трапезникова, А.~Г.~Чурба...",Расчет несжимаемых течений в системе &lt;&lt;п...,Препринты ИПМ им.~М.~В.~Келдыша,2021,None,None,None
ipmp2939,"[101245, 25339, 163334]",:https://doi.org/10.20948/prepr-2021-21,None,None,Препринт,"Б. Г. Фрейнкман, С. В. Поляков, И. О. Толстов,...","Б.~Г.~Фрейнкман, С.~В.~Поляков, И.~О.~Толстов",Расчет основного состояния электрона в неоднор...,Препринты ИПМ им.~М.~В.~Келдыша,2021,None,None,None


In [20]:
print(audb.db.keys())

dict_keys(['113970', '25339', '18171', '22428', '33430', '27667', '29400', '25875', '79709', '37981', '39797', '37852', '53732', '21356', '33208', '138102', '137773', '172205', '127546', '78977', '17922', '9086', '79683', '141571', '39234', '38434', '38371', '180503', '32577', '32578', '62383', '62384', '8657', '19297', '26325', '151447', '23102'])


In [21]:
# auths = []
# for item in audb.db.values():
#     print(type(item))
#     auths.append(pd.DataFrame.from_dict(item,orient='index'))
#     print(auths[-1].info())

In [22]:
# for item in auths:
#     print(type(item))
#     print(item.head())

# Look at papers without abstracts

In [23]:
links_to_check = ['http://mi.mathnet.ru/'+mn_link for mn_link in absdb.db.index[absdb.db['abstract'].isnull()]]

In [24]:
for link, slink in zip(links_to_check, absdb.db.index[absdb.db['abstract'].isnull()]):    
    print(f"year = {pub.loc[slink,'yr']}  link = {link}, name = {pub.loc[slink,'paper']}")


year = 2003  link = http://mi.mathnet.ru/ivm238, name = Разностные схемы для~параболических уравнений на~треугольных сетках
year = 1991  link = http://mi.mathnet.ru/de7543, name = Разностные схемы для задач абсорбционной бистабильности в~полупроводниках
year = 1989  link = http://mi.mathnet.ru/de6914, name = Численные методы для задач стимулирования химических реакций в~газе световым импульсом
year = 1997  link = http://mi.mathnet.ru/mm1370, name = Методика ускорения процесса решения линейной системы уравнений $\alpha$-$\beta$ метода
year = 1992  link = http://mi.mathnet.ru/mm2044, name = О~моделировании задач радиационной газовой динамики на многопроцессорных вычислительных системах
year = 1991  link = http://mi.mathnet.ru/de7540, name = Кинетически-согласованные разностные схемы в~криволинейных системах координат
year = 1988  link = http://mi.mathnet.ru/dan48205, name = Метод расчета двумерных полупроводниковых структур в квазигидродинамическом приближении
year = 1988  link = http://